# IOT Hub Management

This notebook can be used to configure your IOT hub so it can talk to your Qualcom HDK 888 devices.

In [1]:
import sys
sys.path += ['./scripts']

In [2]:
# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [17]:
# %pip install azure-mgmt-iothub
# %pip install azure-iot-device
# %pip install azure-iot-hub


In [33]:
import json
 
config = json.load(open(".azureml/config.json"))

subscription = config['subscription_id']
iot_resource_group = config['iot_resource_group']
iot_hub_name = config['iot_hub_name']
location = config['location']
iot_hub_connection_string = config['iot_hub_connection_string']

In [5]:

from azure.mgmt.iothub import IotHubClient

iothub_client = IotHubClient(
    credential,
    subscription
)

In [6]:
async_iot_hub = iothub_client.iot_hub_resource.begin_create_or_update(
    iot_resource_group,
    iot_hub_name,
    {
        'location': location,
        'subscriptionid': subscription,
        'resourcegroup': iot_resource_group,
        'sku': {
            'name': 'S1',
            'capacity': 2
        }
    }
)

iot_hub = async_iot_hub.result() # Blocking wait for creation

print(iot_hub.name)

MsrSnpeDeviceHub


In [14]:
from utils import Adb

adb = Adb()
devices = adb.get_devices()

print(devices)

['f7a32443']


In [34]:
from azure.iot.hub import IoTHubRegistryManager
from azure.iot.hub.models import CloudToDeviceMethod, CloudToDeviceMethodResult, Twin
from azure.iot.device import IoTHubDeviceClient, MethodResponse

iothub_registry_manager = IoTHubRegistryManager(iot_hub_connection_string)
device = iothub_registry_manager.get_twin('2492867a')

print(device.device_id)

2492867a


In [32]:
# update the device twin to communicate the status of the device.
iothub_registry_manager.update_twin(device.device_id, Twin(tags={'device_type': 'android', 'online': True}))